In [1]:
# import wfdb

# # This will download and extract CDED dataset to './cded'
# wfdb.dl_database('cded', dl_dir='./cded')


Generating record list for: ./Data/Labview/Converted/Labview-v2/s0033DB-v2
Generating record list for: ./Data/Labview/Converted/Labview-v2/s0068DB-v2
Generating record list for: ./Data/Labview/Converted/Labview-v2/s0078DB-v2
Generating record list for: ./Data/Labview/Converted/Labview-v2/s0153DB-v2
Generating record list for: ./Data/Labview/Converted/Labview-v2/s0166DB-v2
Generating record list for: ./Data/Labview/Converted/Labview-v2/s0174DB-v2
Generating record list for: ./Data/Labview/Converted/Labview-v2/s0197DB-v2
Generating record list for: ./Data/Labview/Converted/Labview-v2/s0215DB-v2
Generating record list for: ./Data/Labview/Converted/Labview-v2/s0221DB-v2
Generating record list for: ./Data/Labview/Converted/Labview-v2/s0264DB-v2
Generating record list for: ./Data/Labview/Converted/Labview-v2/s0296DB-v2
Generating record list for: ./Data/Labview/Converted/Labview-v2/s0301DB-v2
Generating record list for: ./Data/Labview/Converted/Labview-v2/s0314DB-v2
Generating record list fo

In [30]:
import pandas as pd
import os

base_path = r'C:/Users/r/Machine_Learning/HRV Early cardiac risk detection/cded/DATA_discription/'

demographics_path = os.path.join(base_path, 'GE-79_Summary_Table-Demographics-MRI-Part1.csv')
labs_path = os.path.join(base_path, 'GE-79_Summary_Table-Labs-BP-Ophthalmogic-Walk.csv')
history_path = os.path.join(base_path, 'GE-79_Summary_Table-MRI-Part5-History.csv')

demographics = pd.read_csv(demographics_path, encoding='latin1')
labs_bp = pd.read_csv(labs_path, encoding='latin1')
history = pd.read_csv(history_path, encoding='latin1')

print("Demographics shape:", demographics.shape)
print("Labs shape:", labs_bp.shape)
print("History shape:", history.shape)


Demographics shape: (121, 254)
Labs shape: (121, 170)
History shape: (121, 174)


In [31]:
demographics.rename(columns={'Patient ID': 'patient_id', 'Visit': 'visit'}, inplace=True)
labs_bp.rename(columns={'patient ID': 'patient_id', 'Visit': 'visit'}, inplace=True)
history.rename(columns={'Patient ID': 'patient_id', 'Visit': 'visit'}, inplace=True)


In [34]:
print(history.columns)
history.rename(columns={'patient ID': 'patient_id', 'visit': 'visit'}, inplace=True)
df = pd.merge(df, history, on=['patient_id', 'visit'], how='left')



Index(['patient ID', 'visit',
       'Perfusion Vascular (Lepto MCA) -  baseline 1 Right - 6 min',
       'Perfusion Vascular (Lepto PCA) -  baseline 1 Right - 6 min',
       'Perfusion Vascular (MCA Perf) - baseline 1 Right - 6 min',
       'Perfusion Vascular (POCA) - baseline 1 Right - 6 min',
       'Perfusion Vascular (whole brain) - baseline 1 whole - 6 min',
       'Perfusion Vascular (ACA Perf) -  baseline 1 whole - 6 min',
       'Perfusion Vascular (ACHA) -  baseline 1 whole - 6 min',
       'Perfusion Vascular (Lepto ACA) -  baseline 1 whole - 6 min',
       ...
       'STATINS', 'ESTROGEN', 'ACE INHIBITORS', 'ARBS', 'BETA BLOCKERS',
       'DIURETICS', 'CA ++ BLOCKERS', 'INSULIN(Yes_or_No)',
       'ORAL HYPOGLYCEMIC', 'OTHER 2'],
      dtype='object', length=174)


In [35]:
df = pd.merge(demographics, labs_bp, on=['patient_id', 'visit'], how='inner')
df = pd.merge(df, history, on=['patient_id', 'visit'], how='left')

print("Merged dataset shape:", df.shape)
print(df[['patient_id', 'visit']].head())


Merged dataset shape: (121, 594)
  patient_id  visit
0      S0030      2
1      S0033      2
2      S0064      2
3      S0068      2
4      S0078      2


In [40]:
print(df['DM, Non-DM, STROKE'].unique())
print(df['HTN or not'].unique())

autonomic_cols = [
    'Dizziness AUTONOMIC SYMPTOMS_x',
    'Syncope AUTONOMIC SYMPTOMS_x',
    'OH AUTONOMIC SYMPTOMS_x'
]
print(df[autonomic_cols].head())


['Non-DM' 'DM']
['ntn' 'HTN' nan]
  Dizziness AUTONOMIC SYMPTOMS_x Syncope AUTONOMIC SYMPTOMS_x  \
0                             NO                           NO   
1                             NO                           NO   
2                             NO                           NO   
3                             NO                           NO   
4                             NO                           NO   

  OH AUTONOMIC SYMPTOMS_x  
0                      NO  
1                      NO  
2                      NO  
3                      NO  
4                      NO  


In [44]:
def define_can_risk(row):
    has_diabetes = row['DM, Non-DM, STROKE'] == 'DM'
    has_htn = row['HTN or not'] == 'HTN'

    autonomic_cols = [
        'Dizziness AUTONOMIC SYMPTOMS_x',
        'Syncope AUTONOMIC SYMPTOMS_x',
        'OH AUTONOMIC SYMPTOMS_x'
    ]
    
    has_autonomic = any(
        str(row.get(col)).strip().upper() == 'YES'
        for col in autonomic_cols if col in row and pd.notna(row[col])
    )

    if has_diabetes and (has_autonomic or has_htn):
        return 1
    else:
        return 0

# Apply the function
df['CAN_risk_label'] = df.apply(define_can_risk, axis=1)

# Check results
print(df['CAN_risk_label'].value_counts())


CAN_risk_label
0    66
1    55
Name: count, dtype: int64


In [43]:
for col in ['Dizziness AUTONOMIC SYMPTOMS_x', 'Syncope AUTONOMIC SYMPTOMS_x', 'OH AUTONOMIC SYMPTOMS_x']:
    print(f"{col} unique values:", df[col].unique())


Dizziness AUTONOMIC SYMPTOMS_x unique values: ['NO' 'Yes' 'YES']
Syncope AUTONOMIC SYMPTOMS_x unique values: ['NO' 'YES' nan]
OH AUTONOMIC SYMPTOMS_x unique values: ['NO' 'YES']


In [45]:
# Drop identifiers and irrelevant columns
features_to_drop = ['patient_id', 'visit', 'DM, Non-DM, STROKE', 'HTN or not', 
                    'Dizziness AUTONOMIC SYMPTOMS_x', 'Syncope AUTONOMIC SYMPTOMS_x', 
                    'OH AUTONOMIC SYMPTOMS_x']

# Keep only numeric features and drop columns we don't need
X = df.drop(columns=features_to_drop + ['CAN_risk_label']).select_dtypes(include=[np.number])

# Target variable
y = df['CAN_risk_label']

print("Feature shape:", X.shape)
print(X.head())


Feature shape: (121, 505)
   Diabetes Duration  HEIGHT (M)  MASS (KG)        BMI  global GM vol  \
0                NaN        1.62       69.4  26.444140         661.50   
1                NaN        1.80       97.0  29.938272         583.40   
2                NaN        1.70       70.9  24.532872         655.58   
3                NaN        1.55       70.8  29.469303         493.01   
4                NaN        1.54       44.0  18.552876         473.03   

   global GM vol covered in template  L superior frontal gyrus (#21) GM  \
0                             661.03                             28.856   
1                             582.94                             25.006   
2                             655.06                             29.075   
3                             491.81                             18.922   
4                             472.70                             19.539   

   R superior frontal gyrus (#22) GM  L middle frontal gyrus (#23) GM  \
0          

In [46]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)


In [47]:
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Random Forest Classifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_scaled, y_train)

# Predictions
y_pred = model.predict(X_test_scaled)

# Evaluation
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


[[19  1]
 [ 2 15]]
              precision    recall  f1-score   support

           0       0.90      0.95      0.93        20
           1       0.94      0.88      0.91        17

    accuracy                           0.92        37
   macro avg       0.92      0.92      0.92        37
weighted avg       0.92      0.92      0.92        37



c:\Users\r\Machine_Learning\HRV Early cardiac risk detection\.venv\lib\site-packages\sklearn\utils\extmath.py:1144: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
c:\Users\r\Machine_Learning\HRV Early cardiac risk detection\.venv\lib\site-packages\sklearn\utils\extmath.py:1149: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
c:\Users\r\Machine_Learning\HRV Early cardiac risk detection\.venv\lib\site-packages\sklearn\utils\extmath.py:1169: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


In [49]:
import wfdb

# Example ECG record name (adjust based on your files)
record_name = 'S0033ECG'  # change to your actual record name

# Path to ECG record folder
ecg_folder = r'C:/Users/r/Machine_Learning/HRV Early cardiac risk detection/cded/Data/ECG/'  # adjust path

# Read the record
record = wfdb.rdrecord(f'{ecg_folder}/{record_name}')

# Access ECG signal data (multi-channel if available)
ecg_signal = record.p_signal  # numpy array, shape=(n_samples, n_channels)

print(f"Signal shape: {ecg_signal.shape}")
print(f"Sampling frequency: {record.fs} Hz")


Signal shape: (595176, 8)
Sampling frequency: 1000 Hz


In [50]:
import neurokit2 as nk

# Example: Take first channel ECG data
ecg_ch1 = ecg_signal[:, 0]

# Detect R-peaks
signals, info = nk.ecg_process(ecg_ch1, sampling_rate=record.fs)

# Extract HRV features
hrv_features = nk.hrv_time(signals["ECG_R_Peaks"], sampling_rate=record.fs)
print(hrv_features)


   HRV_MeanNN   HRV_SDNN  HRV_SDANN1  HRV_SDNNI1  HRV_SDANN2  HRV_SDNNI2  \
0  879.029586  76.276373   76.742899   26.111553   69.216742   39.507039   

   HRV_SDANN5  HRV_SDNNI5  HRV_RMSSD   HRV_SDSD  ...  HRV_IQRNN  HRV_SDRMSSD  \
0         NaN         NaN  20.713638  20.728922  ...     109.75     3.682423   

   HRV_Prc20NN  HRV_Prc80NN  HRV_pNN50  HRV_pNN20  HRV_MinNN  HRV_MaxNN  \
0        821.0        961.0    1.47929  17.307692      766.0     1084.0   

    HRV_HTI  HRV_TINN  
0  8.894737  101.5625  

[1 rows x 25 columns]


In [51]:
import glob
import neurokit2 as nk
import pandas as pd
import wfdb
import os

ecg_folder = r'C:/Users/r/Machine_Learning/HRV Early cardiac risk detection/cded/Data/ECG/'

hrv_list = []

for record_path in glob.glob(os.path.join(ecg_folder, '*ECG.hea')):
    record_name = os.path.splitext(os.path.basename(record_path))[0].replace('.hea','')
    try:
        record = wfdb.rdrecord(os.path.join(ecg_folder, record_name))
        ecg_signal = record.p_signal[:, 0]  # pick first channel
        signals, info = nk.ecg_process(ecg_signal, sampling_rate=record.fs)
        hrv = nk.hrv_time(signals["ECG_R_Peaks"], sampling_rate=record.fs)
        hrv['patient_id'] = record_name[:5]  # adjust slicing based on your naming
        hrv['visit'] = 2  # or extract from metadata if available
        hrv_list.append(hrv)
    except Exception as e:
        print(f"Failed {record_name}: {e}")

df_hrv = pd.concat(hrv_list, ignore_index=True)
print(df_hrv.head())


   HRV_MeanNN    HRV_SDNN  HRV_SDANN1  HRV_SDNNI1  HRV_SDANN2  HRV_SDNNI2  \
0  879.029586   76.276373   76.742899   26.111553   69.216742   39.507039   
1  982.590571  123.302236   84.304607   89.460749   64.294791   88.414396   
2  685.945355   53.959304   59.012233   19.816811   49.785432   32.498280   
3  758.582359  228.805327  196.530270   94.513943  139.704868  130.124056   
4  759.156627  293.123764   96.469767  264.880697  101.417513  265.055038   

   HRV_SDANN5  HRV_SDNNI5   HRV_RMSSD    HRV_SDSD  ...  HRV_Prc20NN  \
0         NaN         NaN   20.713638   20.728922  ...        821.0   
1         NaN         NaN   65.333454   65.414853  ...        956.0   
2         NaN         NaN   14.399222   14.406518  ...        651.0   
3         NaN         NaN  133.358894  133.361227  ...        631.0   
4         NaN         NaN  391.413609  391.807815  ...        583.8   

   HRV_Prc80NN  HRV_pNN50  HRV_pNN20  HRV_MinNN  HRV_MaxNN    HRV_HTI  \
0        961.0   1.479290  17.307692 

In [52]:
df_full = pd.merge(df, df_hrv, on=['patient_id', 'visit'], how='inner')

print("Final dataset shape:", df_full.shape)


Final dataset shape: (42, 620)


In [55]:
# Define columns to drop (non-numeric or identifiers)
features_to_drop = ['patient_id', 'visit', 'DM, Non-DM, STROKE', 'HTN or not', 
                    'Dizziness AUTONOMIC SYMPTOMS_x', 'Syncope AUTONOMIC SYMPTOMS_x', 
                    'OH AUTONOMIC SYMPTOMS_x', 'CAN_risk_label']

# Select numeric features only
X = df_full.drop(columns=features_to_drop).select_dtypes(include=[np.number])

# Target variable
y = df_full['CAN_risk_label']

print(f"Features shape: {X.shape}")
print(f"Target distribution:\n{y.value_counts()}")


Features shape: (42, 530)
Target distribution:
CAN_risk_label
0    23
1    19
Name: count, dtype: int64


In [56]:
print("Original clinical data patient count:", df['patient_id'].nunique())
print("HRV feature patient count:", df_hrv['patient_id'].nunique())
print("Merged dataset patient count:", df_full['patient_id'].nunique())


Original clinical data patient count: 77
HRV feature patient count: 47
Merged dataset patient count: 42


In [57]:
# Left merge clinical data (df) with HRV features (df_hrv)
df_full = pd.merge(df, df_hrv, on=['patient_id', 'visit'], how='left')

print("Merged dataset shape:", df_full.shape)

# Check how many missing HRV feature rows you have
missing_hrv_count = df_full['HRV_MeanNN'].isna().sum()  # example HRV feature column
print(f"Missing HRV feature rows: {missing_hrv_count} out of {df_full.shape[0]}")

# Add a flag indicating if HRV data is available for each row
df_full['hrv_available'] = df_full['HRV_MeanNN'].notna().astype(int)

# Fill missing HRV features with median values
hrv_columns = df_hrv.columns.drop(['patient_id', 'visit'])

for col in hrv_columns:
    median_val = df_full[col].median()
    df_full[col].fillna(median_val, inplace=True)

print("After imputation, missing values per column:")
print(df_full[hrv_columns].isna().sum())

# Now you can prepare features and target for modeling
features_to_drop = ['patient_id', 'visit', 'DM, Non-DM, STROKE', 'HTN or not', 
                    'Dizziness AUTONOMIC SYMPTOMS_x', 'Syncope AUTONOMIC SYMPTOMS_x', 
                    'OH AUTONOMIC SYMPTOMS_x', 'CAN_risk_label']  # plus others if needed

X = df_full.drop(columns=features_to_drop).select_dtypes(include=[np.number])
y = df_full['CAN_risk_label']

print("Final feature shape:", X.shape)
print("Target distribution:\n", y.value_counts())


Merged dataset shape: (121, 620)
Missing HRV feature rows: 79 out of 121
After imputation, missing values per column:
HRV_MeanNN      0
HRV_SDNN        0
HRV_SDANN1      0
HRV_SDNNI1      0
HRV_SDANN2      0
HRV_SDNNI2      0
HRV_SDANN5      0
HRV_SDNNI5      0
HRV_RMSSD       0
HRV_SDSD        0
HRV_CVNN        0
HRV_CVSD        0
HRV_MedianNN    0
HRV_MadNN       0
HRV_MCVNN       0
HRV_IQRNN       0
HRV_SDRMSSD     0
HRV_Prc20NN     0
HRV_Prc80NN     0
HRV_pNN50       0
HRV_pNN20       0
HRV_MinNN       0
HRV_MaxNN       0
HRV_HTI         0
HRV_TINN        0
dtype: int64
Final feature shape: (121, 531)
Target distribution:
 CAN_risk_label
0    66
1    55
Name: count, dtype: int64


C:\Users\r\AppData\Local\Temp\ipykernel_6612\1325462723.py:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_full[col].fillna(median_val, inplace=True)
C:\Users\r\AppData\Local\Temp\ipykernel_6612\1325462723.py:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when 

In [62]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

models = {
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
    'XGBoost': xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42),
    'KNN': KNeighborsClassifier(n_neighbors=5),
    'SVM': SVC(kernel='rbf', probability=True, random_state=42)
}

from sklearn.model_selection import StratifiedKFold, cross_val_score

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

results = {}

for name, model in models.items():
    print(f"Running CV for {name}...")
    pipe = Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler()),
        ('clf', model)
    ])
    scores = cross_val_score(pipe, X, y, cv=skf, scoring='accuracy', n_jobs=-1)
    results[name] = scores
    print(f"{name} CV accuracy scores: {scores}")
    print(f"{name} Mean accuracy: {scores.mean():.4f} (+/- {scores.std():.4f})\n")


Running CV for Random Forest...
Random Forest CV accuracy scores: [0.92       0.79166667 0.875      0.91666667 0.79166667]
Random Forest Mean accuracy: 0.8590 (+/- 0.0572)

Running CV for Logistic Regression...
Logistic Regression CV accuracy scores: [0.84       0.75       0.83333333 0.75       0.70833333]
Logistic Regression Mean accuracy: 0.7763 (+/- 0.0516)

Running CV for XGBoost...
XGBoost CV accuracy scores: [0.88       0.875      0.875      1.         0.95833333]
XGBoost Mean accuracy: 0.9177 (+/- 0.0519)

Running CV for KNN...
KNN CV accuracy scores: [0.6        0.58333333 0.54166667 0.54166667 0.66666667]
KNN Mean accuracy: 0.5867 (+/- 0.0461)

Running CV for SVM...
SVM CV accuracy scores: [0.8        0.75       0.58333333 0.625      0.75      ]
SVM Mean accuracy: 0.7017 (+/- 0.0827)



In [63]:
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xgb
import numpy as np

# Define the XGBoost classifier
xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)

# Parameter grid for RandomizedSearch
param_dist = {
    'n_estimators': np.arange(50, 300, 50),
    'max_depth': np.arange(3, 10),
    'learning_rate': np.linspace(0.01, 0.3, 10),
    'subsample': np.linspace(0.6, 1.0, 5),
    'colsample_bytree': np.linspace(0.6, 1.0, 5),
    'gamma': [0, 0.1, 0.2, 0.3],
    'reg_alpha': [0, 0.01, 0.1],
    'reg_lambda': [1, 1.5, 2]
}

# Randomized search with 5-fold stratified CV
random_search = RandomizedSearchCV(
    estimator=xgb_model,
    param_distributions=param_dist,
    n_iter=50,
    scoring='accuracy',
    cv=5,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

# Fit random search
random_search.fit(X_scaled, y)

print("Best parameters found:")
print(random_search.best_params_)

print(f"Best CV accuracy: {random_search.best_score_:.4f}")


Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best parameters found:
{'subsample': np.float64(0.8), 'reg_lambda': 2, 'reg_alpha': 0.1, 'n_estimators': np.int64(100), 'max_depth': np.int64(4), 'learning_rate': np.float64(0.23555555555555557), 'gamma': 0, 'colsample_bytree': np.float64(0.8)}
Best CV accuracy: 0.9090


c:\Users\r\Machine_Learning\HRV Early cardiac risk detection\.venv\lib\site-packages\xgboost\training.py:183: UserWarning: [04:18:15] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [64]:
import xgboost as xgb
from sklearn.metrics import classification_report, confusion_matrix

# Best params from RandomizedSearchCV (remove np types)
best_params = {
    'subsample': 0.8,
    'reg_lambda': 2,
    'reg_alpha': 0.1,
    'n_estimators': 100,
    'max_depth': 4,
    'learning_rate': 0.23555555555555557,
    'gamma': 0,
    'colsample_bytree': 0.8,
    'random_state': 42,
    'eval_metric': 'logloss'
}

# Initialize model with best params
best_xgb = xgb.XGBClassifier(**best_params)

# Train on training data
best_xgb.fit(X_train_scaled, y_train)

# Predict on test set
y_pred = best_xgb.predict(X_test_scaled)

# Evaluation
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Confusion Matrix:
[[18  2]
 [ 2 15]]

Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.90      0.90        20
           1       0.88      0.88      0.88        17

    accuracy                           0.89        37
   macro avg       0.89      0.89      0.89        37
weighted avg       0.89      0.89      0.89        37

